## 1. 시퀀스
- 문장: 생각이나 감정을 말과 글로 표현할 때 완결된 내용을 나타내는 최소의 단위
- 작문하는 딥러닝 모델 GPT-3: 논리적 일관성 유지, 자연스러움

### 1-1 시퀀스(Sequence)
- 문장, 주가, 날짜, 드라마 등 다양한 유형이 시퀀스 데이터에 포함된다.
- 시퀀스는 데이터에 순서(번호)를 붙여 나열한 것이다. 즉, 순서가 있는 데이터 구조라고 할 수 있다. 그러나 데이터의 순서란 나열 순서일 뿐, 정렬과는 다르다.
- 시퀀스 데이터가 곧 각 요소들의 연관성을 의미하는 것은 아니지만, 우리(인공지능)가 예측을 하려면 어느 정도는 연관성이 있어줘야 한다. 문법이 중요하다.
- 통계에 기반

### 1-2. I 다음 am을 쓰면 반 이상은 맞더라
알바생이 커피를 [ ] 라면 아마도 만든다가 정답이겠죠. 알바생이 커피를 마실 수도 있지만, 통계적으론 만드다.

인공지능이 글을 이해하게 하는 방식도 위와 같습니다. 어떤 문법적인 원리를 통해서가 아니고, 수많은 글을 읽게 함으로써 나는 , 밥을, 그 다음이 먹는다 라는 사실을 알게 하는 거죠. 그런 이유에서 많은 데이터가 곧 좋은 결과를 만들어냅니다.

이 방식을 가장 잘 처리하는 인공지능 중 하나가 **순환신경망(RNN)** 이다.

#### RNN
<start> 입력 -> '나는' 출력 => '나는' 입력 -> '밥을' 출력 => '밥을' 입력 -> '먹었다' 출력 => '먹었다' 입력 -> <end> 출력
    
- 생성한 단어를 다시 입력으로 사용하는 순환적인 특성을 살려 순환신경망이라 한다.
- <end> 와 <start>는 문장 데이터만 있으면 만들어낼 수 있다.

In [11]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


#### 언어모델 (Language Model)
어떤 문구 뒤에 다음 단어가 나올 확률이 높다는 것은 그 다음 단어가 나오는 것이 보다 자연스럽다는 뜻이 됩니다. 그렇다면 '나는' 뒤에 '밥을'이 나오는 것이 자연스럽지 않다는 뜻일까요? 그것은 아닙니다. '나는' 뒤에 올 수 있는 자연스러운 단어의 경우의 수가 워낙 많다 보니 불확실성이 높을 뿐입니다.

n-1 개의 단어 시퀀스 w1, ,,, wn-1가 주어졌을 때, n번째 단어 wn으로 무엇이 올지 예측과는 확률 모델을 언어 모델이라고 부른다.

$P(w_n | w_1, …, w_{n-1};\theta )$

- w1, ,,, wn-1이 주어졌을 때, n번째 단어, wn으로 무엇이 올지 예측하는 구조
- 어떤 텍스트도 언어 모델의 학습 데이터가 될 수 있습니다. n-1번째까지의 단어 시퀀스가 x_train이 되고 n번째 단어가 y_train이 되는 데이터셋
- 이렇게 학습된 언어 모델을 학습 모드가 아닌 테스트 모드로 가동하면, 이 모델은 일정한 단어 시퀀스가 주어진다면 다음 단어, 그 다음 단어를 계속해서 예측해 낼 것

## 2. 데이터 전처리

In [1]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

# 파일을 읽기모드로 열어 봅니다.
file_path = os.getenv('HOME') + '/workplace/aiffel/Exploration/11.Lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 list 형태로 읽어옵니다.

print(raw_corpus[:9])    # 앞에서부터 10라인만 화면에 출력해 볼까요?

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


#### 토큰화
- 텍스트 분류 모델에서 많이 본 것처럼 텍스트 생성 모델에도 단어 사전을 만들게 된다.이때 문장을 일정한 기준으로 쪼개는 과정을 토큰화 과정이라 한다.
- 띄어쓰기 기준으로 나누는 방법
- 문제 케이스
    - Hi, my name is John. *("Hi," "my", …, "john." 으로 분리됨) - 문장부호
    - First, open the first chapter. *(First와 first를 다른 단어로 인식) - 대소문자
    - He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) - 특수문자
- 해결방안
    - "1." 을 막기 위해 문장 부호 양쪽에 공백을 추가 
    - "2." 를 막기 위해 모든 문자들을 소문자로 변환
    - "3." 을 막기 위해 특수문자들은 모두 제거

#### 전처리: 정규표현식(Regex)

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


#### 자연어처리 분야 모델의 입력과 출력
- 모델의 입력이 되는 문장을 소스 문장(Source Sentence)
- 정답 역할을 하게 될 모델의 출력 문장을 타겟 문장(Target Sentence)라고 관례적으로 부른다. 
- 각각 X_train, y_train 에 해당한다
-  위에서 만든 정제 함수를 통해 만든 데이터셋에서 토큰화를 진행한 후 끝 단어 를 없애면 소스 문장, 첫 단어 를 없애면 타겟 문장이 된다.

In [4]:
# 정제 데이터 구축
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

#### tf.keras.preprocessing.text.Tokenizer 패키지
정제된 데이터를 토큰화하고, 단어 사전(vocabulary 또는 dictionary라고 칭함)을 만들어주며, 데이터를 숫자로 변환까지 한 방에 해줍니다. 이 과정을 벡터화(vectorize) 라 하며, 숫자로 변환된 데이터를 텐서(tensor) 라고 칭합니다. 우리가 사용하는 텐서플로우로 만든 모델의 입출력 데이터는 실제로는 모두 이런 텐서로 변환되어 처리되는 것입니다.

#### Tensor
1. Tensor는 배열의 집합이다.
2. 차원의 수는 Rank와 같은말이다.
3. 배열의 차원에따라 불리는 이름이 달라진다.

rank | type    | example

0    | scalar  | [1]

1    | vector  | [1,1]

2    | matrix  | [[1,1],[1,1]]

3    | 3-tensor| [[[1,1],[1,1]],[[1,1], [1,1,]], [1.2], [2,1]]]

n    | n-tensor| 

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0034902210>


In [6]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [7]:
# 텐서 데이터는 모두 정수로 이루어져 있습니다. 이 숫자는 다름 아니라, tokenizer에 구축된 단어 사전의 인덱스
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [8]:
# 생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게 한다
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워넣은 것입니다. 사전에는 없지만 0은 바로 패딩 문자 가 될 것입니다.

- 소스는 2()에서 시작해서 3()으로 끝난 후 0()로 채워져 있다.
- 타겟은 2로 시작하지 않고 소스를 왼쪽으로 한칸 시프트한 형태

#### 데이터셋 객체 생성
텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 tf.data.Dataset 객체 생성
- tf.data.Dataset객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의기능을 제공
- tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

#### 데이터셋 생성 과정 정리
- 정규표현식을 이용한 corpus 생성
- tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
- tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환

## 인공지능 학습
인공지능 = 인공신경망 = 딥러닝 네트워크 = 모델

Embedding (단어를 추상적으로 변환하는 역할) -> 

RNN1 (LSTM) : 문장을 순차적으로 읽으며 단어 간의 연관성 분석 ->

RNN2 (LSTM) : 문장을 순차적으로 읽으며 단어 간의 연관성 분석 -> 

Linear (Dense) : RNN에서 만들어낸 결과를 바탕으로 생성할 단어 결정

####  tf.keras.Model을 Subclassing하는 방식
1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

 입력 텐서에는 단어 사전의 인덱스가 들어 있습니다. Embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔 줍니다. 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용

#### embedding_size 
워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
- 차갑다: [0.0, 1.0]
- 뜨겁다: [1.0, 0.0]
- 미지근하다: [0.5, 0.5]
값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기한다.

#### hidden_size
LSTM 레이어의 hidden state 의 차원수인 hidden_size 도 같은 맥락입니다. hidden_size 는 모델에 얼마나 많은 일꾼을 둘 것인가? 로 이해해도 크게 엇나가지 않습니다. 그 일꾼들은 모두 같은 데이터를 보고 각자의 생각을 가지는데, 역시 충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 갈 뿐 입니다. 이번 실습에는 1024가 적당해보이는군요.

In [11]:
# model build
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 6.70970112e-05, -1.86108082e-04, -3.02427768e-04, ...,
         -2.06798926e-04, -6.12117365e-05, -5.85916969e-05],
        [ 6.38362311e-04, -4.20911456e-05, -3.05910129e-04, ...,
         -3.83489329e-04, -4.51039596e-05, -2.37608227e-04],
        [ 1.01366173e-03,  1.36112561e-04, -3.36070487e-04, ...,
         -5.08250203e-04,  2.73713114e-04, -8.97613703e-04],
        ...,
        [-7.00458558e-03, -1.97080686e-03,  3.24633002e-04, ...,
         -1.74147819e-04,  2.95041199e-03, -8.25500931e-04],
        [-7.43304892e-03, -2.14238628e-03,  3.10141826e-04, ...,
         -1.82372998e-04,  3.13094701e-03, -8.09352205e-04],
        [-7.79099530e-03, -2.28742301e-03,  2.86704104e-04, ...,
         -2.05668461e-04,  3.28538963e-03, -7.93119892e-04]],

       [[ 6.70970112e-05, -1.86108082e-04, -3.02427768e-04, ...,
         -2.06798926e-04, -6.12117365e-05, -5.85916969e-05],
        [ 2.43116869e-04, -2.81393644e-04,  1.

- shape=(256, 20, 7001)
- 7001: Dense 레이어의 출력 차원수
    - 7001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 하기 때문
- 256은 이전 스텝에서 지정한 배치 사이즈
    - take(1)를 통해서 1개의 배치, 즉 256개의 문장 데이터를 가져온 것
- 20: tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한 LSTM 레이어에서 return_sequences=True이라고 지정한 부분에 있습니다. 즉, LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미입니다. 만약 return_sequences=False였다면 LSTM 레이어는 1개의 벡터만 출력했을 것입니다. 그런데 문제는, 우리의 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모른다는 점입니다. 모델을 만들면서 알려준 적도 없습니다. 그럼 20은 언제 알게된 것일까요? 네, 그렇습니다. 데이터를 입력받으면서 비로소 알게 된 것입니다. 우리 데이터셋의 max_len이 20으로 맞춰져 있었던 것입니다.

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


입력 시퀀스의 길이를 모르기 때문에 output shape 를 알려주지 않는다. 그러나 모델의 파라미터 사이즈는 측정된다. 약 22million이다. 

#### 모델 학습

In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 13s 141ms/step - loss: 3.5233
Epoch 2/30
93/93 [==============================] - 13s 135ms/step - loss: 2.8132
Epoch 3/30
93/93 [==============================] - 13s 136ms/step - loss: 2.7256
Epoch 4/30
93/93 [==============================] - 13s 138ms/step - loss: 2.6309
Epoch 5/30
93/93 [==============================] - 12s 130ms/step - loss: 2.5601
Epoch 6/30
93/93 [==============================] - 12s 134ms/step - loss: 2.5113
Epoch 7/30
93/93 [==============================] - 14s 147ms/step - loss: 2.4657
Epoch 8/30
93/93 [==============================] - 13s 139ms/step - loss: 2.4143
Epoch 9/30
93/93 [==============================] - 13s 136ms/step - loss: 2.3663
Epoch 10/30
93/93 [==============================] - 13s 135ms/step - loss: 2.3209
Epoch 11/30
93/93 [==============================] - 12s 133ms/step - loss: 2.2744
Epoch 12/30
93/93 [==============================] - 13s 138ms/step - loss: 2.2302
Epoch 13/30
9

#### 모델 평가
enerate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 합니다.

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

텍스트를 생성하는 함수 안을 들여다보면 while문이 하나 자리잡고 있는 것을 볼 수 있습니다. 왜 그럴까요?

학습 단계에서 우리는 이런 while 문이 필요없었습니다. 소스 문장과 타겟 문장이 있었고, 우리는 소스 문장을 모델에 입력해서 나온 결과를 타겟 문장과 직접 비교하면 그만이었습니다.
그러나 텍스트를 실제로 생성해야 하는 시점에서, 우리에게는 2가지가 없습니다. 하나는 타겟 문장입니다. 또하나는 무엇이냐 하면, 소스 문장입니다. 생각해 보면 우리는 텍스트 생성 태스크를 위해 테스트 데이터셋을 따로 생성한 적이 없습니다.

generate_text() 함수에서 init_sentence를 인자로 받고는 있습니다. 이렇게 받은 인자를 일단 텐서로 만들고 있습니다. 디폴트로는 단어 하나만 받는군요.

   - while의 첫번째 루프에서 test_tensor에 <START> 하나만 들어갔다고 합시다. 우리의 모델이 출력으로 7001개의 단어 중 A를 골랐다고 합시다.
   - while의 두번째 루프에서 test_tensor에는 <START> A가 들어갑니다. 그래서 우리의 모델이 그다음 B를 골랐다고 합시다.
   - while의 세번째 루프에서 test_tensor에는 <START> A B가 들어갑니다. 그래서….. (이하 후략)

In [15]:
# 문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s not prepared to morrow . <end> '